In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import ipywidgets as widgets
import numpy as np

In [15]:
prefix = 'aggressive_4_'
df = pd.read_csv(f'results/{prefix}semstamp_sentence.csv')
df

,P,Num,Origin,aggressive_4_A_normal,aggressive_4_B_normal,aggressive_4_A_flipped,aggressive_4_B_flipped,aggressive_4_A_count,aggressive_4_B_count,aggressive_4_choice,aggressive_4_flipped_choice
0,Evan strolled into the charming café on Rue Cl...,0,A,1.0,0.0,1.0,0.0,1.0,0.0,A,A
1,Evan strolled along the charming streets of Mo...,0,B,0.0,1.0,0.0,1.0,0.0,1.0,B,B
2,Evan strolled into the charming café on Rue Cl...,1,A,1.0,0.0,1.0,0.0,1.0,0.0,A,A
3,Evan strolled along the charming streets of Mo...,1,B,0.0,1.0,0.0,1.0,0.0,1.0,B,B
4,Evan strolled into the charming café on Rue Cl...,2,A,1.0,0.0,1.0,0.0,1.0,0.0,A,A
...,...,...,...,...,...,...,...,...,...,...,...
395,As he wandered through the picturesque streets...,197,B,0.0,1.0,0.0,1.0,0.0,1.0,B,B
396,"Evan slipped into the charming Rue Cler café, ...",198,A,1.0,0.0,1.0,0.0,1.0,0.0,A,A
397,As he wandered through the picturesque streets...,198,B,0.0,1.0,0.0,1.0,0.0,1.0,B,B
398,"Evan ducked into the quaint Rue Cler café, esc...",199,A,1.0,0.0,1.0,0.0,1.0,0.0,A,A


In [16]:
def summary(prefix=""):
    df[f'{prefix}correct'] = (df['Origin'] == df[f'{prefix}choice']).astype(float)
    df[f'{prefix}flipped_correct'] = (df['Origin'] == df[f'{prefix}flipped_choice']).astype(float)
    df[f'{prefix}avg_correct'] = (df[f'{prefix}correct']+df[f'{prefix}flipped_correct'])/2
    print(f"Correct: {df[f'{prefix}correct'].mean()}, Flipped Correct: {df[f'{prefix}flipped_correct'].mean()}, Avg Correct: {df[f'{prefix}avg_correct'].mean()}")

summary(prefix)

Correct: 1.0, Flipped Correct: 1.0, Avg Correct: 1.0


In [17]:
dropdown = widgets.Dropdown(
    options=['avg_correct', 'correct', 'flipped_correct'],
    value='avg_correct',
    description='Statistic:',
)

origin = widgets.Dropdown(
    options=['A', 'B', 'Both'],
    value='Both',
    description='Origin:',
)

slider = widgets.IntSlider(min=1, max=20, step=1, value=1, description='Window:')

def plot(prefix=""):
    match origin.value:
        case 'A':
            data = df[df['Origin'] == 'A'][f'{prefix}{dropdown.value}'].reset_index(drop=True)
            # fall through
        case 'B':
            data = df[df['Origin'] == 'B'][f'{prefix}{dropdown.value}'].reset_index(drop=True)
        case 'Both':
            tmp1 = df[df['Origin'] == 'A'][f'{prefix}{dropdown.value}'].reset_index(drop=True)
            tmp2 = df[df['Origin'] == 'B'][f'{prefix}{dropdown.value}'].reset_index(drop=True)
            data = (tmp1+tmp2)/2
    
    rolling_mean = data.rolling(window=slider.value).mean()
    plt.title(f'{prefix}{dropdown.value} (window={slider.value}, origin={origin.value})')
    plt.xlabel('Mutation number')
    plt.ylabel('Accuracy')
    plt.ylim(-0.1, 1.1)
    plt.yticks([0, 0.5, 1])
    plt.plot(rolling_mean, color='orange')
    plt.show()

widgets.interact(lambda x, y, z: plot(prefix), x=dropdown, y=slider, z=origin)

interactive(children=(Dropdown(description='Statistic:', options=('avg_correct', 'correct', 'flipped_correct')…

<function __main__.<lambda>(x, y, z)>